# Train TextCNN
Stefan/Yuzhao Heng
Since Wed. Mar. 16th. 2022


## Setup
### Ipython



In [1]:
%load_ext autoreload
%autoreload 2


### Colab


In [2]:
import os
import sys


is_on_colab = 'google.colab' in sys.modules

if is_on_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream
    ! pip3 install wfdb
    ! pip3 install pytorch-lightning transformers datasets
    ! pip3 install vit-pytorch

    path = os.path.join('drive', 'My Drive', 'Research', 'ECG Classification', 'ECG-Representation-Learning')
    sys.path.append(path)
    ! ls "{path}"

    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()


import torch
if torch.cuda.is_available():
    ! nvidia-smi


from icecream import ic

from ecg_transformer.util import *
ic(PATH_BASE, DIR_PROJ, PKG_NM)



ic| PATH_BASE: '/Users/stefanh/Documents/UMich/Research/ECG Classification'
    DIR_PROJ: 'ECG-Representation-Learning'
    PKG_NM: 'ecg_transformer'


('/Users/stefanh/Documents/UMich/Research/ECG Classification',
 'ECG-Representation-Learning',
 'ecg_transformer')

### code


In [3]:
from pytorch_lightning.utilities.seed import seed_everything
from icecream import ic

from ecg_transformer.models.train import get_all_setup

seed_everything(config('random-seed'))



Global seed set to 77


77

## Prep

In [4]:
# model_size = 'debug'
model_size = 'base'
t = 'original'

# n_sample = 4
n_sample = None

train_args = dict(
    num_train_epoch=64,
    train_batch_size=256,
    eval_batch_size=256*2,
    warmup_ratio=0.1,
    n_sample=n_sample,
    precision=16,
    log_per_epoch=True,
    # log_to_console=False
    save_while_training=True,
    save_every_n_epoch=8,
    save_top_k=4
)
model, trainer = get_all_setup(model_size=model_size, train_args=train_args, ptbxl_type=t)



2022-04-05 19:11:22| [Get PTB-XL splits]::get_ptbxl_splits::ptb_dataset.py:98, INFO - Getting PTB-XL splits with n=17792... 
2022-04-05 19:11:22| [Get PTB-XL splits]::get_ptbxl_splits::ptb_dataset.py:111, INFO - Splits created with sizes {train: 4, eval: 4, test: 4}... 


## Train



In [5]:
trainer.train()



2022-04-05 19:11:22| [EcgVit Train]::train::train.py:100, INFO - Launched training model EcgVitConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "intermediate_size": 1024,
  "max_signal_length": 2560,
  "num_attention_heads": 8,
  "num_channels": 12,
  "num_class": 12,
  "num_hidden_layers": 2,
  "patch_size": 64,
  "transformers_version": "4.17.0"
}
 with args {
  "num_train_epoch": 64,
  "train_batch_size": 256,
  "eval_batch_size": 512,
  "learning_rate": 0.0003,
  "weight_decay": 0.1,
  "warmup_ratio": 0.1,
  "n_sample": 4,
  "patience": 8,
  "precision": 16,
  "log_per_epoch": true,
  "log_to_console": true,
  "save_while_training": true,
  "save_every_n_epoch": 8,
  "save_top_k": 4,
  "steps_per_epoch": 1,
  "n_step": 64,
  "output_dir": "/Users/stefanh/Documents/UMich/Research/ECG Classification/ECG-Representation-Learning/models/2022-04-05_19-11-22"
}
 and {model: {name: EcgVit, input shape: 12 x 2560, #patch: 40}, #epoch: 64,

/Users/stefanh/opt/anaconda3/envs/ecg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:658: UserWarning: You passed `Trainer(accelerator='cpu', precision=16)` but native AMP is not supported on CPU. Using `precision='bf16'` instead.
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type   | Params
---------------------------------
0 | model | EcgVit | 1.8 M 
---------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.217     Total estimated model params size (MB)
/Users/stefanh/opt/anaconda3/envs/ecg/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the

2022-04-05 19:11:23| [EcgVit Train]::log::train.py:162, INFO - {epoch:  0/64, eval/loss: 0.7326, eval/binary_accuracy: 48.94, eval/weighted_binary_accuracy: 52.14, eval/binary_negative_recall:  3.42, eval/binary_positive_recall:  97.1, eval/macro_auc: 29.17, eval/per_class_auc: {NORM:   0.00, IMI: 100.00, ABQRS:  25.00, SR:  50.00, AFIB:   0.00, AFLT:   0.00}}


Global seed set to 77
/Users/stefanh/opt/anaconda3/envs/ecg/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


2022-04-05 19:11:24| [EcgVit Train]::log::train.py:162, INFO - {epoch:  1/64, step:  1/64, train/learning_rate: 0.000e+00, train/loss: 0.7439, train/binary_accuracy: 47.54, train/weighted_binary_accuracy: 46.04, train/binary_negative_recall:   2.7, train/binary_positive_recall: 96.32, train/macro_auc: -, train/per_class_auc: {}}
2022-04-05 19:11:26| [EcgVit Train]::log::train.py:162, INFO - {epoch:  1/64, eval/loss: 0.7326, eval/binary_accuracy: 48.94, eval/weighted_binary_accuracy: 52.14, eval/binary_negative_recall:  3.42, eval/binary_positive_recall:  97.1, eval/macro_auc: 29.17, eval/per_class_auc: {NORM:   0.00, IMI: 100.00, ABQRS:  25.00, SR:  50.00, AFIB:   0.00, AFLT:   0.00}}
2022-04-05 19:11:27| [EcgVit Train]::log::train.py:162, INFO - {epoch:  2/64, step:  2/64, train/learning_rate: 5.000e-05, train/loss: 0.7284, train/binary_accuracy:  50.7, train/weighted_binary_accuracy: 53.05, train/binary_negative_recall:  3.55, train/binary_positive_recall:  97.2, train/macro_auc: -, 

/Users/stefanh/opt/anaconda3/envs/ecg/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## Evaluate



In [6]:
# TODO

